In [23]:
# Import Pandas Numpy and OS

import pandas as pd
import numpy as np 
import os

print('Imports complete - pandas ready to use')


Imports complete - pandas ready to use


In [22]:
# Create an empty dictionary to hold the match ratings
# This will look like, a count for each  match rating
# MatchRatings = {'-1' : 10, '0' : 20, '1' : 10}

MatchRatings = {}
print(MatchRatings)


{}


In [2]:
# Create DF from All-Seasons.csv
try:
    all_seasons = pd.read_csv(r'data-csv\All-Seasons.csv', parse_dates=["Date"], dayfirst=True)
    print('All Seasons DataFrame created successfully')
except:
    print('Something went wrong - check file is not open in Excel or make sure you have run the imports')
    quit()

# sort by Date highest to lowest
all_seasons.sort_values(by=['Date'], ascending=False, inplace=True)


All Seasons DataFrame created successfully


In [28]:
# create functions to use in the calculations of the goal ratings

def print_row():
     print(index, ' : ', row['Date'], row['HomeTeam'], row['AwayTeam'], row['FTHG'], row['FTAG'], row['FTR'])
     return

In [29]:
# loop through each match in all_seasons dataframe

for index, row in all_seasons.iterrows():

    DT = row['Date']
    HT = row['HomeTeam']
    AT = row['AwayTeam']
    RE = row['FTR']
    
    print_row()
    
    # print(index, ' : ', row['Date'], row['HomeTeam'], row['AwayTeam'], row['FTHG'], row['FTAG'], row['FTR'])
    
    break


0  :  2021-03-10 00:00:00 Man City Southampton 5 2 H


Select each game in turn

For the home team select their last 6 games
Superior Goal Rating = goals scored (HSG) / goals conceded (HCG)

For the away team select their last 6 games
Superior Goal Rating = goals scored (ASG) / goals conceded (ACG)

Calculate the Match Rating = (HSG - HCG) - (ASG - ACG)

The Matching Rating should be added to the MatchRating{} dictionary
If there is an existing MatchRating then increment its count


In [8]:
# Current form for home team

current_form = all_seasons.loc[ (all_seasons.Date <= DT) & ((all_seasons.HomeTeam == HT) | (all_seasons.AwayTeam == HT)) ].head(6)

current_form


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR
0,2021-03-10,Man City,Southampton,5,2,H,3.0,1.0,H
5,2021-03-07,Man City,Man United,0,2,A,0.0,1.0,A
17,2021-03-02,Man City,Wolves,4,1,H,1.0,0.0,H
24,2021-02-27,Man City,West Ham,2,1,H,1.0,1.0,D
32,2021-02-21,Arsenal,Man City,0,1,A,0.0,1.0,A
40,2021-02-17,Everton,Man City,1,3,A,1.0,1.0,D


In [16]:
# HOME TEAM
# goals scored / goals conceded for the last 6 games 

Scored = 0
Scored = current_form.loc[((current_form.HomeTeam == HT)), 'FTHG'].sum()
Scored = Scored + (current_form.loc[((current_form.AwayTeam == HT)), 'FTAG'].sum())

Conceded = 0
Conceded = current_form.loc[((current_form.HomeTeam == HT)), 'FTAG'].sum()
Conceded = Conceded + (current_form.loc[((current_form.AwayTeam == HT)), 'FTHG'].sum())

HomeSuperiorRating = Scored - Conceded

print('Scored : %d \t Conceded : %d \t Goal Superiority Rating : %d'%(Scored, Conceded, HomeSuperiorRating))


Scored : 5 	 Conceded : 2 	 Goal Superiority Rating : 3


In [11]:
# Current form for AWAY team

current_form = all_seasons.loc[ (all_seasons.Date <= DT) & ((all_seasons.HomeTeam == AT) | (all_seasons.AwayTeam == AT)) ].head(6)

current_form

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR
0,2021-03-10,Man City,Southampton,5,2,H,3.0,1.0,H
8,2021-03-06,Sheffield United,Southampton,0,2,A,0.0,1.0,A
18,2021-03-01,Everton,Southampton,1,0,H,1.0,0.0,H
28,2021-02-23,Leeds,Southampton,3,0,H,0.0,0.0,D
34,2021-02-20,Southampton,Chelsea,1,1,D,1.0,0.0,H
43,2021-02-14,Southampton,Wolves,1,2,A,1.0,0.0,H


In [13]:
# AWAY TEAM
# goals scored / goals conceded for the last 6 games 

Scored = 0
Scored = current_form.loc[((current_form.HomeTeam == AT)), 'FTHG'].sum()
Scored = Scored + (current_form.loc[((current_form.AwayTeam == AT)), 'FTAG'].sum())

Conceded = 0
Conceded = current_form.loc[((current_form.HomeTeam == AT)), 'FTAG'].sum()
Conceded = Conceded + (current_form.loc[((current_form.AwayTeam == AT)), 'FTHG'].sum())

AwaySuperiorRating = Scored - Conceded

print('Scored : %d \t Conceded : %d \t Goal Superiority Rating : %d'%(Scored, Conceded, AwaySuperiorRating))


Scored : 6 	 Conceded : 12 	 Goal Superiority Rating : -6


In [17]:
# Match Rating

MatchRating = HomeSuperiorRating - AwaySuperiorRating

print(' Match Rating :', MatchRating)


 Match Rating : 9


In [27]:
# If the MatchRating doesn't exist in the dictionary create it and give it a value of 0
# then increment it
# if a MatchRating does exist then just increment it

MatchRatings[MatchRating] = MatchRatings.get(MatchRating, 0) + 1

MatchRatings


{9: 4}